In [64]:
#@title Imports & Set Path
import pandas as pd
from random import randint
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Dense, LSTM
from keras.layers import Add, Average 
from keras.layers import Dropout, Activation, Input, Flatten
from keras.layers import TimeDistributed, Bidirectional
from keras import optimizers
from keras import regularizers
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings('ignore')

path = 'Data/' #add the path of your data

In [65]:
#@title reading from google colab drive 
from google.colab import drive 
drive.mount('/content/gdrive')

path = '/content/gdrive/My Drive/ChallengeUP-master/ChallengeUP-master/Data/'

base = pd.read_csv(path + 'CompleteDataSet_training_competition.csv')
base.loc[base.Tag==20,'Tag'] = 12

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [66]:
#@title Split per second
def split_per_second(dataFrames=None,sensors=None,getLables=True):

  higher_length_sample = 0   
  
  dataFrames_splited_per_second = []

  labels_splited_per_second = np.zeros((1,12))

  cont=0
  current_index=0
  
  dataFrame = dataFrames
  
  teste = dataFrame['TimeStamps'].apply(
      lambda x: x.replace(microsecond=0))
  
  teste = teste.drop_duplicates()
  
  for y in range(len(teste)):
    tDataFrame = dataFrame.iloc[current_index:(current_index+25),:] 
    
    tempDataFrame = tDataFrame.loc[
        tDataFrame['TimeStamps']
        .apply(lambda x: x.replace(microsecond=0))
        .isin(teste.iloc[[y]])
    ]
    
    
    if(tempDataFrame.shape[0] == 0):
      print("ZERO:")
      print(y,teste[y])
      print(tempDataFrame.head())
      continue
        
    
    if higher_length_sample < len(tempDataFrame):
      higher_length_sample = len(tempDataFrame)
      
    current_index+=len(tempDataFrame)
    
    if getLables == True:
      label_mode = tempDataFrame['Tag'].mode()


      if label_mode.shape[0] > 1:
        size=(len(tempDataFrame['Tag'])//2)
        df_new1 = tempDataFrame[:size]
        df_new2 = tempDataFrame[size:]
        
        df_new1 = np.asarray(df_new1.iloc[:,sensors])
        dataFrames_splited_per_second.append(df_new1)
        df_new2 = np.asarray(df_new2.iloc[:,sensors])
        dataFrames_splited_per_second.append(df_new2)
        
        cont+=1
        
                
      else:
        tempDataFrame = np.asarray(tempDataFrame.iloc[:,sensors])
        dataFrames_splited_per_second.append(tempDataFrame)
      
      
      label_mode = np.asarray(label_mode)-1
      label_mode = to_categorical(label_mode,num_classes=12)

      labels_splited_per_second = np.append(
          labels_splited_per_second,label_mode,axis=0)
        
    else:
      tempDataFrame = np.asarray(tempDataFrame.iloc[:,sensors])
      dataFrames_splited_per_second.append(tempDataFrame)
    
    cont+=1
      
  dataFrames_splited_per_second = np.asarray(dataFrames_splited_per_second)
  
  if getLables == True:
    labels_splited_per_second = np.delete(labels_splited_per_second,0,0)
  
  if getLables == True:
    return dataFrames_splited_per_second,labels_splited_per_second,higher_length_sample
  else:
    return dataFrames_splited_per_second, higher_length_sample 

sensors = [
    1,2,3, #accelerometer ankle
    #4,5,6, #gyroscope ankle
    #7, #luminosity ankle
    
    #8,9,10 #accelerometer right pocket
    #11,12,13 #gyroscope right pocket
    #14, #luminosity right pocket
    
    #15,16,17 # accelerometer belt
    #18,19,20, # gyroscope belt
    #21, #luminosity belt
    
    22,23,24, # accelerometer neck
    #25,26,27, # gyroscope neck
    #28, #luminosity neck
    
    29,30,31 # accelerometer wrist
    #32,33,34, # gyroscope wrist
    ##35, #luminosity wrist
    
    #36 #brain sensor
]
   




In [67]:
base.head(5)

,TimeStamps,AnkleAccelerometer x-axis,AnkleAccelerometer y-axis,AnkleAccelerometer z-axis,AnkleAngularVelocity x-axis,AnkleAngularVelocity y-axis,AnkleAngularVelocity z-axis,AnkleLuminosity illuminance,RightPocketAccelerometer x-axis,RightPocketAccelerometer y-axis,RightPocketAccelerometer z-axis,RightPocketAngularVelocity x-axis,RightPocketAngularVelocity y-axis,RightPocketAngularVelocity z-axis,RightPocketLuminosity illuminance,BeltAccelerometer x-axis,BeltAccelerometer y-axis,BeltAccelerometer z-axis,BeltAngularVelocity x-axis,BeltAngularVelocity y-axis,BeltAngularVelocity z-axis,BeltLuminosity illuminance,NeckAccelerometer x-axis,NeckAccelerometer y-axis,NeckAccelerometer z-axis,NeckAngularVelocity x-axis,NeckAngularVelocity y-axis,NeckAngularVelocity z-axis,NeckLuminosity illuminance,WristAccelerometer x-axis,WristAccelerometer y-axis,WristAccelerometer z-axis,WristAngularVelocity x-axis,WristAngularVelocity y-axis,WristAngularVelocity z-axis,WristLuminosity illuminance,BrainSensor,Infrared1,Infrared2,Infrared3,Infrared4,Infrared5,Infrared6,Subject,Trial,Activity,Tag
0,2018-07-04T12:04:17.738369,-1.005,0.229,-0.083,-0.671,0.488,-2.683,0.0,-0.981,0.260,0.098,1.768,-1.890,0.732,0.0,0.146,0.895,0.367,-1.463,-3.841,-2.622,392.633,-0.672,0.072,-0.815,-3.293,-7.805,9.207,1231.336,-1.240,-0.557,0.227,-75.854,44.024,315.915,2108.154,45,1,1,1,1,1,1,1,1,1,7
1,2018-07-04T12:04:17.790509,-1.005,0.228,-0.082,-3.415,-0.549,0.122,0.0,-0.981,0.260,0.098,3.354,-2.500,-0.061,0.0,0.146,0.895,0.367,1.037,-4.024,-1.768,392.633,-0.672,0.072,-0.815,-9.756,-8.049,5.122,1231.336,-1.240,-0.557,0.227,-28.780,55.549,344.146,2108.154,1,1,1,1,1,1,1,1,1,1,7
2,2018-07-04T12:04:17.836632,-1.005,0.231,-0.079,-2.622,-1.402,-0.549,0.0,-0.975,0.282,0.091,3.720,-2.500,-0.793,0.0,0.178,0.896,0.373,0.366,-2.500,-0.488,392.633,-0.705,0.084,-0.779,-17.866,-18.354,4.329,1231.336,-1.884,-0.687,0.251,27.256,87.134,372.012,2108.154,325,1,1,1,1,1,1,1,1,1,7
3,2018-07-04T12:04:17.885262,-1.005,0.231,-0.079,-2.561,-2.195,-1.220,0.0,-0.973,0.301,0.103,-1.951,-1.159,-1.280,0.0,0.160,0.895,0.372,0.854,1.951,1.585,392.633,-0.727,0.095,-0.758,-18.537,-10.061,0.366,1231.336,-1.884,-0.687,0.251,2.866,117.683,356.463,2108.154,396,1,1,1,1,1,1,1,1,1,7
4,2018-07-04T12:04:17.945423,-1.008,0.229,-0.072,-3.537,-2.073,-0.305,0.0,-0.973,0.301,0.103,-5.366,-1.280,-0.732,0.0,0.160,0.895,0.372,2.012,4.634,3.354,392.633,-0.727,0.095,-0.758,4.878,2.866,-3.232,1231.336,-1.597,-0.646,0.112,10.000,30.671,126.768,2108.154,-436,1,1,1,1,1,1,1,1,1,7


In [68]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160736 entries, 0 to 160735
Data columns (total 47 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   TimeStamps                         160736 non-null  object 
 1   AnkleAccelerometer x-axis          160736 non-null  float64
 2   AnkleAccelerometer y-axis          160736 non-null  float64
 3   AnkleAccelerometer z-axis          160736 non-null  float64
 4   AnkleAngularVelocity x-axis        160736 non-null  float64
 5   AnkleAngularVelocity y-axis        160736 non-null  float64
 6   AnkleAngularVelocity z-axis        160736 non-null  float64
 7   AnkleLuminosity illuminance        160736 non-null  float64
 8   RightPocketAccelerometer x-axis    160736 non-null  float64
 9   RightPocketAccelerometer y-axis    160736 non-null  float64
 10  RightPocketAccelerometer z-axis    160736 non-null  float64
 11  RightPocketAngularVelocity x-axis  1607

In [69]:
base.loc[base["Tag"] == 12]

# test = base["Tag"]

# test = pd.DataFrame(data = test,columns=["Tag"])
# test.loc[12]

,TimeStamps,AnkleAccelerometer x-axis,AnkleAccelerometer y-axis,AnkleAccelerometer z-axis,AnkleAngularVelocity x-axis,AnkleAngularVelocity y-axis,AnkleAngularVelocity z-axis,AnkleLuminosity illuminance,RightPocketAccelerometer x-axis,RightPocketAccelerometer y-axis,RightPocketAccelerometer z-axis,RightPocketAngularVelocity x-axis,RightPocketAngularVelocity y-axis,RightPocketAngularVelocity z-axis,RightPocketLuminosity illuminance,BeltAccelerometer x-axis,BeltAccelerometer y-axis,BeltAccelerometer z-axis,BeltAngularVelocity x-axis,BeltAngularVelocity y-axis,BeltAngularVelocity z-axis,BeltLuminosity illuminance,NeckAccelerometer x-axis,NeckAccelerometer y-axis,NeckAccelerometer z-axis,NeckAngularVelocity x-axis,NeckAngularVelocity y-axis,NeckAngularVelocity z-axis,NeckLuminosity illuminance,WristAccelerometer x-axis,WristAccelerometer y-axis,WristAccelerometer z-axis,WristAngularVelocity x-axis,WristAngularVelocity y-axis,WristAngularVelocity z-axis,WristLuminosity illuminance,BrainSensor,Infrared1,Infrared2,Infrared3,Infrared4,Infrared5,Infrared6,Subject,Trial,Activity,Tag
54181,2018-07-06T12:44:03.859625,0.917,0.807,-0.649,39.451,95.732,-30.061,230.415,-2.350,-0.433,0.146,-14.268,59.878,21.098,1.777,-0.243,-0.298,-0.343,7.073,87.439,3.354,8.058,-2.794,0.491,-2.483,-1.220,50.122,16.707,366.939,-1.501,-0.260,-0.549,-93.780,92.134,-19.146,117.919,-4,1,1,1,1,0,0,7,1,2,12
54182,2018-07-06T12:44:03.905754,0.316,0.703,0.226,156.342,33.537,-78.659,230.415,-0.302,0.241,-0.283,33.720,-0.793,31.402,1.777,-0.243,-0.298,-0.343,-16.890,52.134,30.427,8.058,-1.821,0.344,-0.778,-12.134,-52.988,30.122,366.939,-0.415,-0.513,-0.892,406.463,203.293,304.268,117.919,-45,1,1,1,1,0,0,7,1,2,12
54183,2018-07-06T12:44:03.952880,0.316,0.703,0.226,-58.963,-11.037,-40.061,230.415,-0.208,-0.066,0.184,1.280,1.098,18.537,1.777,-0.128,-0.145,0.106,-4.878,-17.805,28.537,8.058,-1.821,0.344,-0.778,17.500,-95.610,47.317,366.939,-0.415,-0.513,-0.892,723.293,171.890,269.817,117.919,50,1,1,1,1,0,0,7,1,2,12
54184,2018-07-06T12:44:04.004021,-0.146,0.017,-0.040,63.780,-49.878,-54.085,230.415,-0.208,-0.066,0.184,-0.915,-5.610,6.646,2.251,0.027,-0.393,0.001,-24.207,6.585,33.049,8.058,-0.216,-0.117,0.025,20.000,26.890,-25.549,547.100,-0.216,0.639,0.185,489.512,105.915,14.634,163.655,234,1,1,1,1,0,0,7,1,2,12
54185,2018-07-06T12:44:04.047137,-0.268,0.324,-0.892,191.159,-25.793,-68.354,230.415,-0.395,-0.174,0.240,24.573,-9.146,-8.293,2.251,0.027,-0.393,0.001,-11.402,-2.195,15.000,8.058,-0.041,0.042,-0.216,-2.744,13.476,-22.683,547.100,-0.216,0.639,0.185,78.293,145.732,-16.585,163.655,44,1,1,1,1,0,0,7,1,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108602,2018-07-10T11:34:42.375084,0.051,-0.891,-0.437,-0.488,0.488,0.244,281.646,-0.998,0.020,0.267,-2.439,-2.683,0.183,5.214,-0.090,-0.929,-0.400,0.000,-0.732,0.671,17.065,0.001,0.048,-1.060,-0.915,-4.390,0.000,864.009,0.847,-0.050,-0.381,-0.671,-0.854,-0.610,463.530,2047,1,1,1,1,1,0,12,3,2,12
108603,2018-07-10T11:34:42.435246,0.047,-0.891,-0.434,0.000,0.366,0.061,281.646,-0.994,0.022,0.272,-0.854,-1.463,-1.098,5.214,-0.088,-0.930,-0.398,0.366,-0.427,1.341,17.065,0.004,0.032,-1.062,4.024,-21.768,1.037,864.009,0.850,-0.048,-0.385,-0.122,0.000,0.061,463.530,971,1,1,1,1,1,0,12,3,2,12
108604,2018-07-10T11:34:42.486382,0.047,-0.891,-0.434,-0.610,0.366,-0.122,281.646,-0.999,0.017,0.267,-0.915,-1.768,-0.610,5.214,-0.088,-0.930,-0.398,0.610,-0.610,0.854,17.065,0.004,0.032,-1.062,4.085,-14.939,0.305,864.009,0.852,-0.051,-0.389,-0.183,-0.671,0.244,463.530,82,1,1,1,1,1,0,12,3,2,12
108605,2018-07-10T11:34:42.536521,0.053,-0.894,-0.436,-0.305,0.549,-0.061,281.646,-0.999,0.017,0.267,-0.122,-1.098,-0.854,5.214,-0.087,-0.928,-0.399,0.854,-0.732,1.098,17.065,-0.007,0.026,-1.067,0.000,-7.866,-2.195,864.009,0.852,-0.051,-0.389,0.183,0.122,0.427,463.530,2047,1,1,1,1,1,0,12,3,2,12


In [70]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160736 entries, 0 to 160735
Data columns (total 47 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   TimeStamps                         160736 non-null  object 
 1   AnkleAccelerometer x-axis          160736 non-null  float64
 2   AnkleAccelerometer y-axis          160736 non-null  float64
 3   AnkleAccelerometer z-axis          160736 non-null  float64
 4   AnkleAngularVelocity x-axis        160736 non-null  float64
 5   AnkleAngularVelocity y-axis        160736 non-null  float64
 6   AnkleAngularVelocity z-axis        160736 non-null  float64
 7   AnkleLuminosity illuminance        160736 non-null  float64
 8   RightPocketAccelerometer x-axis    160736 non-null  float64
 9   RightPocketAccelerometer y-axis    160736 non-null  float64
 10  RightPocketAccelerometer z-axis    160736 non-null  float64
 11  RightPocketAngularVelocity x-axis  1607

In [71]:
base['TimeStamps'] = pd.to_datetime(base['TimeStamps'])
# base.head()
# AnkleAccelXDf = base[["TimeStamps",'AnkleAccelerometer x-axis',"Tag"]]
# AnkleAccelXDf = pd.DataFrame(data= AnkleAccelXDf, columns=["TimeStamps","AnkleAccelerometer x-axis", "Tag"])
# AnkleAccelXDf.head()

In [72]:
train_dataFrames_splited_per_second,Y_train,higher_length_train = split_per_second(base,sensors,True)

train_dataFrames_splited_per_second,test_dataFrames_splited_per_second, Y_train, Y_test = train_test_split(train_dataFrames_splited_per_second,Y_train,test_size=0.25,random_state=10)
higher_length = higher_length_train

In [73]:
train_dataFrames_splited_per_second[[0]]

array([array([[-1.106e+00, -4.060e-01, -3.640e-01, -2.199e+00, -7.200e-02,
        -1.589e+00, -3.150e+00, -1.153e+00,  5.700e-02],
       [-1.106e+00, -4.060e-01, -3.640e-01, -2.214e+00, -4.700e-02,
        -2.400e+00, -3.150e+00, -1.153e+00,  5.700e-02],
       [-1.441e+00, -4.600e-02, -2.600e-01, -8.790e-01,  3.700e-02,
        -7.630e-01, -1.502e+00, -4.410e-01,  3.720e-01],
       [-9.240e-01, -1.770e-01, -3.250e-01, -8.790e-01,  3.700e-02,
        -7.630e-01,  4.160e-01,  1.620e-01,  1.730e-01],
       [-9.240e-01, -1.770e-01, -3.250e-01,  3.220e-01,  1.250e-01,
         3.760e-01,  4.160e-01,  1.620e-01,  1.730e-01],
       [-2.360e-01, -5.330e-01, -1.580e-01, -5.900e-02, -1.300e-02,
         4.900e-02,  4.920e-01, -7.100e-02, -2.180e-01],
       [-3.600e-01, -1.610e-01,  1.290e-01, -5.900e-02, -1.300e-02,
         4.900e-02, -4.800e-02, -8.000e-02, -1.330e-01],
       [-1.615e+00,  4.040e-01, -4.950e-01,  4.000e-02,  2.300e-02,
        -6.800e-02, -4.800e-02, -8.000e-02, -1.330

In [74]:
train_dataFrames_splited_per_second.shape

(6890,)

In [75]:
base.columns

Index(['TimeStamps', 'AnkleAccelerometer x-axis', 'AnkleAccelerometer y-axis',
       'AnkleAccelerometer z-axis', 'AnkleAngularVelocity x-axis ',
       'AnkleAngularVelocity y-axis ', 'AnkleAngularVelocity z-axis ',
       'AnkleLuminosity illuminance ', 'RightPocketAccelerometer x-axis ',
       'RightPocketAccelerometer y-axis ', 'RightPocketAccelerometer z-axis ',
       'RightPocketAngularVelocity x-axis',
       'RightPocketAngularVelocity y-axis',
       'RightPocketAngularVelocity z-axis',
       'RightPocketLuminosity illuminance', 'BeltAccelerometer x-axis',
       'BeltAccelerometer y-axis', 'BeltAccelerometer z-axis',
       'BeltAngularVelocity x-axis', 'BeltAngularVelocity y-axis',
       'BeltAngularVelocity z-axis', 'BeltLuminosity illuminance',
       'NeckAccelerometer x-axis', 'NeckAccelerometer y-axis',
       'NeckAccelerometer z-axis', 'NeckAngularVelocity x-axis',
       'NeckAngularVelocity y-axis', 'NeckAngularVelocity z-axis',
       'NeckLuminosity illuminan

In [76]:

x = [col for col in base.columns]
x


['TimeStamps',
 'AnkleAccelerometer x-axis',
 'AnkleAccelerometer y-axis',
 'AnkleAccelerometer z-axis',
 'AnkleAngularVelocity x-axis ',
 'AnkleAngularVelocity y-axis ',
 'AnkleAngularVelocity z-axis ',
 'AnkleLuminosity illuminance ',
 'RightPocketAccelerometer x-axis ',
 'RightPocketAccelerometer y-axis ',
 'RightPocketAccelerometer z-axis ',
 'RightPocketAngularVelocity x-axis',
 'RightPocketAngularVelocity y-axis',
 'RightPocketAngularVelocity z-axis',
 'RightPocketLuminosity illuminance',
 'BeltAccelerometer x-axis',
 'BeltAccelerometer y-axis',
 'BeltAccelerometer z-axis',
 'BeltAngularVelocity x-axis',
 'BeltAngularVelocity y-axis',
 'BeltAngularVelocity z-axis',
 'BeltLuminosity illuminance',
 'NeckAccelerometer x-axis',
 'NeckAccelerometer y-axis',
 'NeckAccelerometer z-axis',
 'NeckAngularVelocity x-axis',
 'NeckAngularVelocity y-axis',
 'NeckAngularVelocity z-axis',
 'NeckLuminosity illuminance',
 'WristAccelerometer x-axis',
 'WristAccelerometer y-axis',
 'WristAcceleromet

In [77]:
base.head(1)

,TimeStamps,AnkleAccelerometer x-axis,AnkleAccelerometer y-axis,AnkleAccelerometer z-axis,AnkleAngularVelocity x-axis,AnkleAngularVelocity y-axis,AnkleAngularVelocity z-axis,AnkleLuminosity illuminance,RightPocketAccelerometer x-axis,RightPocketAccelerometer y-axis,RightPocketAccelerometer z-axis,RightPocketAngularVelocity x-axis,RightPocketAngularVelocity y-axis,RightPocketAngularVelocity z-axis,RightPocketLuminosity illuminance,BeltAccelerometer x-axis,BeltAccelerometer y-axis,BeltAccelerometer z-axis,BeltAngularVelocity x-axis,BeltAngularVelocity y-axis,BeltAngularVelocity z-axis,BeltLuminosity illuminance,NeckAccelerometer x-axis,NeckAccelerometer y-axis,NeckAccelerometer z-axis,NeckAngularVelocity x-axis,NeckAngularVelocity y-axis,NeckAngularVelocity z-axis,NeckLuminosity illuminance,WristAccelerometer x-axis,WristAccelerometer y-axis,WristAccelerometer z-axis,WristAngularVelocity x-axis,WristAngularVelocity y-axis,WristAngularVelocity z-axis,WristLuminosity illuminance,BrainSensor,Infrared1,Infrared2,Infrared3,Infrared4,Infrared5,Infrared6,Subject,Trial,Activity,Tag
0,2018-07-04 12:04:17.738369,-1.005,0.229,-0.083,-0.671,0.488,-2.683,0.0,-0.981,0.26,0.098,1.768,-1.89,0.732,0.0,0.146,0.895,0.367,-1.463,-3.841,-2.622,392.633,-0.672,0.072,-0.815,-3.293,-7.805,9.207,1231.336,-1.24,-0.557,0.227,-75.854,44.024,315.915,2108.154,45,1,1,1,1,1,1,1,1,1,7


In [78]:
# Ankletest = split_per_second(AnkleAccelXDf,sensors=sensors,getLables=True)


In [79]:
# Ankletest[0][2]

In [80]:
# train_dataFrames_splited_per_second
# train_adjusted = pd.DataFrame(data= train_dataFrames_splited_per_second, columns = ["test"])
# train_adjusted.iloc[0]

Y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [81]:
base.loc[base['Tag'].isin([8])].count()

TimeStamps                           28911
AnkleAccelerometer x-axis            28911
AnkleAccelerometer y-axis            28911
AnkleAccelerometer z-axis            28911
AnkleAngularVelocity x-axis          28911
AnkleAngularVelocity y-axis          28911
AnkleAngularVelocity z-axis          28911
AnkleLuminosity illuminance          28911
RightPocketAccelerometer x-axis      28911
RightPocketAccelerometer y-axis      28911
RightPocketAccelerometer z-axis      28911
RightPocketAngularVelocity x-axis    28911
RightPocketAngularVelocity y-axis    28911
RightPocketAngularVelocity z-axis    28911
RightPocketLuminosity illuminance    28911
BeltAccelerometer x-axis             28911
BeltAccelerometer y-axis             28911
BeltAccelerometer z-axis             28911
BeltAngularVelocity x-axis           28911
BeltAngularVelocity y-axis           28911
BeltAngularVelocity z-axis           28911
BeltLuminosity illuminance           28911
NeckAccelerometer x-axis             28911
NeckAcceler

In [82]:
#@title Pad data

def match_data(data=None,index=0,final_len=25):
  
  if(len(data)>final_len):
    print("Error!")
    return(-1)
  
  final_len = final_len-len(data)
  
  for x in range(final_len):
    i = index - x
    vector = data[i]
    vector = np.reshape(vector,(1,vector.shape[0]))
    data = np.concatenate((data[:i],vector,data[i:]),axis=0)

  return data

def completeData(X=None,final_len=22):
  
  new_dataFrames_splited_by_second = []
  for x in range(len(X)):
    test = match_data(
        data=X[x],
        index=len(X[x])-1,
        final_len=higher_length
    )
    new_dataFrames_splited_by_second.append(test)
  
  new_dataFrames_splited_by_second = np.asarray(
      new_dataFrames_splited_by_second)
  
  #print(new_dataFrames_splited_by_second.shape)
  #shape = (len(X),final_len,X[0].shape[1])
  #X = np.reshape(X,shape)
  
  return new_dataFrames_splited_by_second


############################
X_train = completeData(train_dataFrames_splited_per_second,final_len=higher_length)
X_test = completeData(test_dataFrames_splited_per_second,final_len=higher_length)
print("Train:",X_train.shape,Y_train.shape)
print("Test:",X_test.shape,Y_test.shape)
#print(X.shape,Y.shape)

Train: (6890, 22, 9) (6890, 12)
Test: (2297, 22, 9) (2297, 12)


In [83]:
X_train.shape

(6890, 22, 9)

In [84]:
Y_train.shape

(6890, 12)

In [85]:
for element in X_train[0]:
  print(element)

[-1.106 -0.406 -0.364 -2.199 -0.072 -1.589 -3.15  -1.153  0.057]
[-1.106 -0.406 -0.364 -2.214 -0.047 -2.4   -3.15  -1.153  0.057]
[-1.441 -0.046 -0.26  -0.879  0.037 -0.763 -1.502 -0.441  0.372]
[-0.924 -0.177 -0.325 -0.879  0.037 -0.763  0.416  0.162  0.173]
[-0.924 -0.177 -0.325  0.322  0.125  0.376  0.416  0.162  0.173]
[-0.236 -0.533 -0.158 -0.059 -0.013  0.049  0.492 -0.071 -0.218]
[-0.36  -0.161  0.129 -0.059 -0.013  0.049 -0.048 -0.08  -0.133]
[-1.615  0.404 -0.495  0.04   0.023 -0.068 -0.048 -0.08  -0.133]
[-1.615e+00  4.040e-01 -4.950e-01 -8.410e-01 -1.130e-01 -7.750e-01
 -9.280e-01 -1.000e-03  1.650e-01]
[-1.469 -0.282 -0.226 -2.459 -0.178 -2.191 -3.517 -0.544 -0.167]
[-1.134 -0.361 -0.349 -1.802 -0.181 -1.73  -2.627 -0.562  0.534]
[-1.504  0.214 -0.21  -0.408 -0.023 -0.106 -0.279 -0.169  0.283]
[-0.407 -0.594 -0.782 -0.408 -0.023 -0.106  0.55  -0.064 -0.125]
[-0.407 -0.594 -0.782 -0.408 -0.023 -0.106  0.55  -0.064 -0.125]
[-0.091  0.178  0.35   0.251  0.07   0.339  0.55  -0.

In [86]:
for element in Y_train[1]:
  print(element)

0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [87]:
# X_train = X_train.reshape(6890, 198)

In [88]:
X_train.shape

(6890, 22, 9)

In [89]:
#@title Callback class to print metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class MetricsPrinter(keras.callbacks.Callback):
  def __init__(self):
    self.train_accs = []
    self.train_precisions = []
    self.train_recalls = []
    self.train_f1_scores = []
    self.test_accs = []
    self.test_precisions = []
    self.test_recalls = []
    self.test_f1_scores = []
  
  def on_epoch_end(self, epoch, logs={}):  
    train_predictions = model.predict(X_train)
    y_train_category=[np.argmax(v) for v in Y_train]
    y_train_predict_category=[np.argmax(v) for v in train_predictions]
    
    test_predictions = model.predict(X_test)
    y_test_category=[np.argmax(v) for v in Y_test]
    y_test_predict_category=[np.argmax(v) for v in test_predictions]
    
    # ------
    
    train_acc = accuracy_score(
        y_train_category, y_train_predict_category)
    
    train_precision = precision_score(
        y_train_category, y_train_predict_category, average='micro')
    
    train_recall = recall_score(
        y_train_category, y_train_predict_category, average='micro')
    
    train_f1 = f1_score(
        y_train_category, y_train_predict_category, average='micro')
    
    test_acc = accuracy_score(
        y_test_category, y_test_predict_category)
    
    test_precision = precision_score(
        y_test_category, y_test_predict_category, average='micro')
    
    test_recall = recall_score(
        y_test_category, y_test_predict_category, average='micro')
    
    test_f1 = f1_score(
        y_test_category, y_test_predict_category, average='micro')
    
    # ------
    
    self.train_accs.append(train_acc)
    self.train_precisions.append(train_precision)
    self.train_recalls.append(train_recall)
    self.train_f1_scores.append(train_f1)

    self.test_accs.append(test_acc)
    self.test_precisions.append(test_precision)
    self.test_recalls.append(test_recall)
    self.test_f1_scores.append(test_f1)
    
    # ------
    
    print ('train_acc: %.4f\ttrain_precision: %.4f\t\ttrain_recall: %.4f\ttrain_f1_score: %.4f' 
           % (train_acc, train_precision, train_recall, train_f1))
    
    print ('test_acc: %.4f\ttest_precision: %.4f\t\ttest_recall: %.4f\ttest_f1_score: %.4f' 
           % (test_acc, test_precision, test_recall, test_f1))
    
    print ('----------------------------------------------------------')

In [90]:
# from imblearn.over_sampling import SMOTE
# from sklearn.utils import class_weight
# smote = SMOTE()

# X_train, Y_train = smote.fit_resample(X_train,Y_train)
# X_train.shape



In [91]:
# X_train =X_train.reshape(23940,22,9)

In [92]:
#@title Create the model


# class_weight = class_weight.comput_class_weight("Balanced",np.unique(Y_train))

keras.backend.clear_session()
input_shape = X_train.shape[1:]
print(input_shape)


model = Sequential()


activation_Dense='softmax' 
units = 400
#LSTM MODEL



model.add(Bidirectional(LSTM(units=units,
                                  return_sequences=True,
                                 input_shape=input_shape)))

model.add(Dropout(rate=0.1))

model.add(Bidirectional(LSTM(units=units,
                                  return_sequences=True)))

model.add(Dropout(rate=0.1))

model.add(Bidirectional(LSTM(units=units,
                                  return_sequences=True)))

model.add(Dropout(rate=0.1))

model.add(Bidirectional(LSTM(units=units,
                                  return_sequences=True)))







model.add(Flatten())

model.add(Dense(12,activation=activation_Dense))


adam = optimizers.Adam(lr=0.001, 
                       beta_1=0.9, 
                       beta_2=0.999, 
                       epsilon=None, 
                       decay=0.0, 
                       amsgrad=False)

model.compile(loss = 'categorical_crossentropy',
              optimizer = "adam")

metricsPrinter = MetricsPrinter()



(22, 9)


In [93]:
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

number_of_epochs = 10000 #35
earlyStopping = EarlyStopping(monitor="val_loss",patience = 5, 
                              verbose = 1)

mcp_save = ModelCheckpoint("/content/gdrive/My Drive/ChallengeUP-master/ChallengeUP-master/Data/3boost_test1.h5",
                           save_best_only = True
                           ,monitor = "val_loss", verbose =1)


model.fit(
  X_train,
  Y_train,
  epochs=number_of_epochs,
  batch_size=32, 
  verbose=1, 
  validation_split = 0.3,
  shuffle=True,
  callbacks=[metricsPrinter,earlyStopping,mcp_save]
)

plt.figure(figsize=(10,5), dpi=100)
plt.grid(True, axis='y', linestyle='--', linewidth=0.5)


plt.plot(metricsPrinter.train_f1_scores, 'bo-', linewidth=3)
plt.plot(metricsPrinter.test_f1_scores, 'ro-', linewidth=3)
plt.ylabel('Percentage')
plt.xlabel('Epoch')
plt.legend(['Train F1-score',
            'Validation F1-score'])

plt.show()
  

Epoch 1/10000
110/151 [====================>.........] - ETA: 4s - loss: 1.5945

KeyboardInterrupt: ignored

In [94]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 22, 800)           1312000   
_________________________________________________________________
dropout (Dropout)            (None, 22, 800)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 22, 800)           3843200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 800)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 22, 800)           3843200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 22, 800)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 22, 800)           3

In [ ]:
# load a saved model
from keras.models import load_model

model = load_model("/content/gdrive/My Drive/ChallengeUP-master/ChallengeUP-master/Data/3boost_test1.h5")

In [ ]:
from sklearn.metrics import confusion_matrix

train_predictions = model.predict(X_train)
y_train_category=[np.argmax(v) for v in Y_train]
y_train_predict_category=[np.argmax(v) for v in train_predictions]
    
test_predictions = model.predict(X_test)
y_test_category=[np.argmax(v) for v in Y_test]
y_test_predict_category=[np.argmax(v) for v in test_predictions]


labels=[1,2,3,4,5,6,7,8,9,10,11,20]

cnf_matrix1 = confusion_matrix(y_train_category, y_train_predict_category)
confusion_ma1 = pd.DataFrame(cnf_matrix1, columns=labels, index=labels)

cnf_matrix2 = confusion_matrix(y_test_category, y_test_predict_category)
confusion_ma2 = pd.DataFrame(cnf_matrix2, columns=labels, index=labels)

In [ ]:
#@title Read test set

base_test = pd.read_csv(path + 'CompleteDataSet_testing_competition.csv')

base_test['TimeStamps'] = pd.to_datetime(base_test['TimeStamps'])
base_test = base_test.iloc[1:] 
real_test_splited_per_second, higher_length_real_test = split_per_second(
    base_test,sensors,False)



In [ ]:
#@title Generating the final results
X_real_test = completeData(X=real_test_splited_per_second,
                           final_len=higher_length)

print("Test size:",len(X_real_test))
X_real_test2 = np.asarray(X_real_test).astype("float32")
real_test_predictions = model.predict(X_real_test2)

y_predict_real=[np.argmax(t) for t in real_test_predictions]

y_predict_real = np.asarray(y_predict_real)+1

y_predict_real = [20 if prediction==12 else prediction for prediction 
                  in y_predict_real]

test_real = base_test['TimeStamps'].apply(
    lambda x: x.replace(microsecond=0))

test_real = test_real.drop_duplicates()

test_real = pd.to_datetime(test_real, 
                           format='%d-%b-%Y %H:%M:%S', 
                           utc=True)

print(test_real.shape[0],len(y_predict_real))

d = {'timestamp': test_real, 'target': y_predict_real}
final_results = pd.DataFrame(data=d)
final_results = final_results[['timestamp','target']]
print(final_results.head(100))


Test size: 3060
3060 3060
                     timestamp  target
1    2018-07-11 17:48:46+00:00       7
15   2018-07-11 17:48:47+00:00       1
34   2018-07-11 17:48:48+00:00       2
52   2018-07-11 17:48:49+00:00      11
71   2018-07-11 17:48:50+00:00      11
...                        ...     ...
1580 2018-07-11 17:52:53+00:00      11
1599 2018-07-11 17:52:54+00:00      11
1617 2018-07-11 17:52:55+00:00      11
1620 2018-07-11 17:53:25+00:00       4
1634 2018-07-11 17:53:26+00:00      11

[100 rows x 2 columns]


In [ ]:
print(final_results.head())

                   timestamp  target
1  2018-07-11 17:48:46+00:00       7
15 2018-07-11 17:48:47+00:00       1
34 2018-07-11 17:48:48+00:00       2
52 2018-07-11 17:48:49+00:00      11
71 2018-07-11 17:48:50+00:00      11


In [ ]:
#@title Submission File

!pip install -U -q PyDrive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import datetime

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
driveWriter = GoogleDrive(gauth)

folder_id = '1El1JO73c9fdMmqnVkB-8dryE47oyoJXP'
file_title = 'test-results' + str(datetime.datetime.now()) + "3boost_test1" + '.csv' 

submission_file = driveWriter.CreateFile({
    'title': file_title,
    'parents': [{'kind': 'drive#fileLink', 'id': folder_id}]
})

submission_file.SetContentString(
    final_results.to_csv(
        index=False, columns=['timestamp', 'target']
    )
)

submission_file.Upload()